In [66]:
import pdftotext
import glob
import string
import re
import os
from tqdm.notebook import tqdm
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.corpora import Dictionary
from gensim.models import LdaModel
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import pandas as pd

C:\Users\maggi\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [67]:
# This is using the same set of pdfs as was fetched in the fasttext demo

#pdfs = glob.glob('data/*.pdf')
txts = glob.glob('../data/texts/*text.txt')

C:\Users\maggi\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


First, grab the text from the pdfs and do some minor cleaning.

In [84]:
#mm
txts

C:\Users\maggi\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['../data/texts\\PG100_text.txt',
 '../data/texts\\PG101_text.txt',
 '../data/texts\\PG102_text.txt',
 '../data/texts\\PG103_text.txt',
 '../data/texts\\PG104_text.txt',
 '../data/texts\\PG105_text.txt',
 '../data/texts\\PG106_text.txt',
 '../data/texts\\PG107_text.txt',
 '../data/texts\\PG108_text.txt',
 '../data/texts\\PG109_text.txt',
 '../data/texts\\PG110_text.txt',
 '../data/texts\\PG111_text.txt',
 '../data/texts\\PG112_text.txt',
 '../data/texts\\PG113_text.txt',
 '../data/texts\\PG115_text.txt',
 '../data/texts\\PG116_text.txt',
 '../data/texts\\PG117_text.txt',
 '../data/texts\\PG118_text.txt',
 '../data/texts\\PG119_text.txt',
 '../data/texts\\PG120_text.txt',
 '../data/texts\\PG121_text.txt',
 '../data/texts\\PG122_text.txt',
 '../data/texts\\PG123_text.txt',
 '../data/texts\\PG124_text.txt',
 '../data/texts\\PG125_text.txt',
 '../data/texts\\PG126_text.txt',
 '../data/texts\\PG127_text.txt',
 '../data/texts\\PG128_text.txt',
 '../data/texts\\PG130_text.txt',
 '../data/text

In [71]:
docs = []

for filename in tqdm(txts):
    with open(filename, 'r', encoding = 'utf8') as f:
        # added encoding = 'utf8' to prevent a continuation byte error
        txt = f
        #pdf = pdftotext.PDF(f)
        txt = '\n\n'.join(txt).lower()
        txt = re.sub('-\s+', '', txt)
        txt = ' '.join(re.findall('[a-z\-]{2,}', txt))
        docs.append(txt)

C:\Users\maggi\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:8: DeprecationWarning: invalid escape sequence \s
<>:9: DeprecationWarning: invalid escape sequence \-
<>:8: DeprecationWarning: invalid escape sequence \s
<>:9: DeprecationWarning: invalid escape sequence \-
<ipython-input-71-4a63d78b2d6f>:8: DeprecationWarning: invalid escape sequence \s
  txt = re.sub('-\s+', '', txt)
<ipython-input-71-4a63d78b2d6f>:9: DeprecationWarning: invalid escape sequence \-
  txt = ' '.join(re.findall('[a-z\-]{2,}', txt))


  0%|          | 0/333 [00:00<?, ?it/s]

The next step is preprocessing. This will involve tokenizing, removing numbers and lemmatizing.

In [73]:
# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
for idx in tqdm(range(len(docs))):
    docs[idx] = docs[idx].lower()  # Convert to lowercase.
    docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

# Remove numbers, but not words that contain numbers.
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]

# Remove words that are only one character.
docs = [[token for token in doc if len(token) > 1] for doc in docs]

#take out the lemmatizer for now
lemmatizer = WordNetLemmatizer()
docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

C:\Users\maggi\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\maggi\Anaconda3\lib\asyncio\events.py:81: DeprecationWarning: `run_cell_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  self._context.run(self._callback, *self._args)


  0%|          | 0/333 [00:00<?, ?it/s]

Then build a gensim Dictionary to hold the documents and filter about the infrequent words and super common ones.

In [74]:
# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.5)

C:\Users\maggi\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Then use the dictionary to convert from the documents to a bad-of-words representation.

In [75]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

C:\Users\maggi\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\maggi\Anaconda3\lib\asyncio\events.py:81: DeprecationWarning: `run_cell_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  self._context.run(self._callback, *self._args)


In [76]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 15210
Number of documents: 333


C:\Users\maggi\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Finally, train an LDA model.

In [77]:
# Set training parameters.
num_topics = 6
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

C:\Users\maggi\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Take the results of the model and export the pyLDAvis as an html.

In [78]:
vis = gensimvis.prepare(model, corpus, dictionary, mds = 'mmds', sort_topics=False)
#added mds = 'mmds' to prevent a complex number error
pyLDAvis.save_html(vis, 'lda.html')

C:\Users\maggi\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Build a DataFrame showing the topic distribution of each document.

In [79]:
contents = []

for txt, doc in tqdm(zip(txts, docs)):
    bow = dictionary.doc2bow(doc)
    topics = model.get_document_topics(bow)
    for res in topics:
        topic, pct = res
        contents.append({'txt': txt, 'topic': topic, 'percent': pct})

topics = pd.DataFrame(contents)

C:\Users\maggi\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0it [00:00, ?it/s]

Pull out the five most representative documents for each topic.

In [80]:
for i in range(6):
    for j in range(5):

        file = topics[topics.topic == i].sort_values('percent', ascending = False).head().iloc[j, 0]

        cmd = f'cp {file} representative_docs/topic_{i + 1}/.'

        os.system(cmd)

C:\Users\maggi\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [81]:
topics

C:\Users\maggi\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,txt,topic,percent
0,../data/texts\PG100_text.txt,1,0.011075
1,../data/texts\PG100_text.txt,2,0.986273
2,../data/texts\PG101_text.txt,1,0.040456
3,../data/texts\PG101_text.txt,3,0.101645
4,../data/texts\PG101_text.txt,4,0.841651
...,...,...,...
924,../data/texts\PG99_text.txt,1,0.162421
925,../data/texts\PG99_text.txt,3,0.672867
926,../data/texts\PG9_text.txt,2,0.015396
927,../data/texts\PG9_text.txt,3,0.972876


In [82]:
contents

C:\Users\maggi\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[{'txt': '../data/texts\\PG100_text.txt', 'topic': 1, 'percent': 0.011074662},
 {'txt': '../data/texts\\PG100_text.txt', 'topic': 2, 'percent': 0.9862729},
 {'txt': '../data/texts\\PG101_text.txt', 'topic': 1, 'percent': 0.040455583},
 {'txt': '../data/texts\\PG101_text.txt', 'topic': 3, 'percent': 0.10164499},
 {'txt': '../data/texts\\PG101_text.txt', 'topic': 4, 'percent': 0.84165114},
 {'txt': '../data/texts\\PG101_text.txt', 'topic': 5, 'percent': 0.016240953},
 {'txt': '../data/texts\\PG102_text.txt', 'topic': 0, 'percent': 0.36074272},
 {'txt': '../data/texts\\PG102_text.txt', 'topic': 1, 'percent': 0.32969126},
 {'txt': '../data/texts\\PG102_text.txt', 'topic': 4, 'percent': 0.30639926},
 {'txt': '../data/texts\\PG103_text.txt', 'topic': 0, 'percent': 0.7118817},
 {'txt': '../data/texts\\PG103_text.txt', 'topic': 1, 'percent': 0.11123849},
 {'txt': '../data/texts\\PG103_text.txt', 'topic': 3, 'percent': 0.15992393},
 {'txt': '../data/texts\\PG103_text.txt', 'topic': 5, 'percent'